## Try and estimate viral fraction of pig gut metaG:
We have pig gut metagenomes, from which a certain part is most probably viral. Can we estimate what % of the metaG is viral using sourmash? Do these estimations agree with the read mapping estimations? 
How do these percentages differ at scale 1000 vs 100 (they don't).



## Sourmash is able to find the fraction but not the proper sequences
While sourmash is able to estimate the portion of the metaG that is composed of viral sequences, it does not do a good job in finding which viral sequences. Maybe skipmers will make this more viable, would like to test on 50 metaGs or so: These ones should already be sketched. However, would like to compare to read mapping so would have to be the same sequences 

Read mapping was done to 95% deduplicated viral sequences. How would we compare to skipmers?
- build skipmer database of the skipm2n3 for 95% ANI dedup vOTUs (singleton)
- make skipmer signatures for 50 metaGs that have been read-mapped as well: readmap_pig.txt has these 
- run manysearch or fastgather

Reads are already sketched, into 1 giant file. So are the vOTUs (for taxonomy)


### So skipmers are not improving number of viruses found by much:
How much of the content in the db is exactly similar? Maybe thats why it picks up fragments of all these small pieces of virus. Use sourmash pairwise, should show similarity within db. Then use betterplot to plot the similarity.
k-size too small? Maybe compare kmers and larger skipmers too. Biggest skipmer i have is 27. 

In [ ]:
mamba activate branchwater-skipmer

srun --account=ctbrowngrp -p med2 -J skipm -t 2:00:00 -c 40 --mem=100gb --pty bash

sourmash scripts pairwise \
pig_votus.skipmer.zip -m skipm2n3 -k 21 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.skipm2n3.k21.csv \
-c 40 --write-all

sourmash scripts pairwise \
pig_votus.skipmer.zip -m skipm2n3 -k 27 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.skipm2n3.k27.csv \
-c 40 --write-all

sourmash scripts pairwise \
pig_votus.skipmer.zip -m skipm2n3 -k 24 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.skipm2n3.k24.csv \
-c 40 --write-all

sourmash scripts pairwise \
hq_virseqs.singleton.95.zip -m DNA -k 51 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.dna.k51.csv \
-c 40 --write-all

sourmash scripts pairwise \
hq_virseqs.singleton.95.zip -m DNA -k 31 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.dna.k31.csv \
-c 40 --write-all

sourmash scripts pairwise \
hq_virseqs.singleton.95.zip -m DNA -k 21 \
-s 100 -t 0.5 -o ../pairwise/pairwise_votus.dna.k21.csv \
-c 40 --write-all

In [ ]:
# needs some mem
srun --account=ctbrowngrp -p med2 -J skipm -t 1:00:00 -c 2 --mem=50gb --pty bash
# plotting the pairwse results:
# convert pairwise
# may need to do the write all
sourmash scripts pairwise_to_matrix pairwise_votus.skipm2n3.k27.csv \
-o pairwise_votus.skipm2n3.k27.cmp \
--labels-to pairwise_votus.skipm2n3.k27.labels_to.csv
    
# plot!
sourmash scripts plot2 pairwise_votus.skipm2n3.k27.cmp \
pairwise_votus.skipm2n3.k27.labels_to.csv \
-o plots/plot2.votus.skipm2n3.k27.png

In [ ]:
srun --account=ctbrowngrp -p med2 -J sketch -t 5:00:00 -c 1 --mem=50gb --pty bash

# make one merged sketch for all vOTU 
mamba activate branchwater-skipmer

# Sketch the viral contigs
sourmash sketch dna \
-p k=21,scaled=100,k=31,scaled=100,k=51,scaled=100 --singleton \
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/vOTUs/hq_virseqs.95.cluster.fa \
-o hq_virseqs.singleton.95.zip

In [ ]:
# Srun for more mem
srun --account=ctbrowngrp -p med2 -J skipm -t 5:00:00 -c 12 --mem=50gb --pty bash

# make one merged sketch for all vOTU 
mamba activate branchwater-skipmer

sourmash scripts fastmultigather ../skipmer_manysearch/reads.50.skipmer.zip \
pig_votus.skipmer -c 12 \
-o votus95.skipm2n3.k21s100.csv -m skipm2n3 -k 21 -s 100 -t 200

#fastgather?

In [ ]:
# viral sequence locations:
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/vOTUs/all_viral_sequences.fa
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/vOTUs/hq_virseqs.95.cluster.fa

# DNA level comparison, normal kmers
- Sketching DNA, one concatenated sketch for all contigs 

In [ ]:
# Srun for more mem
srun --account=ctbrowngrp -p med2 -J manysearch -t 1:00:00 -c 32 --mem=50gb --pty bash

# make one merged sketch for all vOTU 
mamba activate branchwater-skipmer

# Sketch the viral contigs
sourmash sketch dna \
-p k=21,scaled=100,k=31,scaled=100 \
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/vOTUs/hq_virseqs.95.cluster.fa \
--name hq_virseq_95 -o hq_virseqs.95.merge.zip

# manysearch the viral contigs x the sigs of the reads (fp.txt contains filepaths of read sigs)
sourmash scripts manysearch hq_virseqs.95.merge.zip \
fp.txt -o virfrac.k21s100.highqual_95.338.csv -k 21 -s 100 -t 0

# Read mapping using bowtie2
- Create a bt2 database file
- Use snakemake (see workflow/readmap.smk) for the read mapping part
- CoverM to get number of reads that mapped

In [ ]:
# compare this to read mapping using bowtie2
# create a bt2 file for mapping
# see readmap.smk
srun --account=ctbrowngrp -p med2 -J bt2 -t 5:00:00 -c 40 --mem=50gb --pty bash

mamba activate bowtie2
bowtie2-build \
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/vOTUs/all_viral_sequences.fa \
all_viral_seq.bt2 -p 40 --large-index

# only the 95% dedup, high quality shit
bowtie2-build \
/group/ctbrowngrp2/scratch/annie/2024-pigparadigm/results/vOTUs/hq_virseqs.95.cluster.fa \
hq_virseqs.95.bt2 -p 40 --large-index

# run snakefile
snakemake --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 40 --rerun-incomplete -k -s readmap.smk -n

# coverM with reads?
mamba activate coverm
coverm contig -m mean \
-b *.bam -o readmap_vir_t0.mean.tsv &> coverm_t0.out.txt

## Viromes:
- want to see if higher percent of content is estimated to be viral if using virome
- Picked 3 SRAs that are virome acc to their metadata
- Download sra, convert to reads
- Sketch (DNA, skipmer)
- manysearch

In [ ]:
# get a virome, or 5. 
# download reads (SRA download)
# fq dump
# sketch
srun --account=ctbrowngrp -p med2 -J virome -t 5:00:00 -c 40 --mem=100gb --pty bash
# run manysearch x viral contigs
SRR7293098
SRR7287117
SRR7288018

conda activate sra
aws s3 cp --quiet --no-sign-request \
s3://sra-pub-run-odp/sra/SRR7287117/SRR7287117 SRR7287117
aws s3 cp --quiet --no-sign-request \
s3://sra-pub-run-odp/sra/SRR7288018/SRR7288018 SRR7288018

In [ ]:
# fq dump
for f in SRR*
do
fasterq-dump $f --threads 32 \
-O reads/$f --skip-technical \
--bufsize 1000MB --curcache 10000MB
done

# sketch
for f in SRR*
do
sourmash sketch dna \
-p k=21,scaled=100,k=31,scaled=100 \
"$f/${f}_1.fastq" "$f/${f}_2.fastq" \
--name "$f" -o "$f.zip"
done

# skip sketch
sourmash scripts manysketch -p skipm2n3,k=21,scaled=100 \
-c 40 fp.reads.txt -o reads_virome.skipmer.zip

# skip search
sourmash scripts manysearch pig_votus.skipmer.concat.zip -c 40 \
fp_manysearch.txt -o viral_fraction.virome.k21.skipm2n3.csv -m skipm2n3 -k 21 -s 100 -t 0

In [ ]:
#manysearch
# Does manysearch not work with zips? 

sourmash scripts manysearch hq_virseqs.95.merge.zip \
fp.txt -o virtest.k21s100.338.csv -k 21 -s 100 -t 0

In [ ]:
# Normal search works, why not manysearch??
sourmash search \
hq_virseqs.95.merge.zip \
/group/ctbrowngrp2/amhorst/2025-pigparadigm/results/metag_sig/ERR1135219.zip \
 -k 21 --scaled 100 -o test.csv

## Skipmer 
- Use skipmers see if works better

In [ ]:
# skipmers were already sketched
# sketch 50 reads. 
# Can we do it for 4?
srun --account=ctbrowngrp -p med2 -J virome -t 1:00:00 -c 40 --mem=60gb --pty bash

mamba activate branchwater-skipmer

# sketch 5 metaGs in skipmer
# DO NOT SKETCH SINGLETON
sourmash scripts manysketch -p skipm2n3,k=21,k=24,k=27,abund,scaled=100 \
-c 40 50_readsketch.csv -o reads.50.abund.skipmer.zip

# manysearch
sourmash scripts manysearch pig_votus.skipmer.concat.zip -c 40 \
reads.50.skipmer.zip -o viral_fraction.50.k21.skipm2n3.csv -m skipm2n3 -k 21 -s 100 -t 0

In [ ]:
mamba activate branchwater-skipmer

# instead of manysearch: fastgather?
sourmash scripts fastgather ../../metag_signatures/ERR3211942.zip \
/group/ctbrowngrp2/amhorst/2025-vsmash/results/pig_votus/sketches/pig_votus.dnakmer.zip \
-o ERR3211942.csv -k 21 -s 100 -t 10